In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import torch
from torch.autograd import Variable
from torch import Tensor
import io
import requests
from torch import nn


tqdm().pandas()
%matplotlib notebook
%run utils.py

In [ ]:
N, D_in, H1,H2, D_out = 100, 36, 100,100, 1
model = nn.Sequential(
    nn.Linear(D_in, H1, bias=False),
    nn.Tanh(),
    nn.Linear(H1, H2, bias=False),
    nn.Tanh(),
    nn.Linear(H1, D_out, bias=False),
)

In [47]:
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-4

num_epochs = 50
with tqdm(total=num_epochs*134900) as pbar:
    for epoch in range(num_epochs):
        df_iter = pd.read_csv('data/yellow_tripdata_2009-01_cleaned.csv', chunksize=10000)
        total_loss = 0
        count = 0
        for df in df_iter:
            x, y = df_to_torch(df)
            pbar.update(1)
            # Forward pass: compute predicted y by passing x to the model. Module objects
            # override the call operator so you can call them like functions. When
            # doing so you pass a Variable of input data to the Module and it produces
            # a Variable of output data.
            y_pred = model(x)

            # Compute and print loss. We pass Variables containing the predicted and true
            # values of y, and the loss function returns a Variable containing the loss.
            loss = loss_fn(y_pred, y)
            total_loss += loss.data[0]
            count += 1
            if count % 100 == 0:
                print(total_loss/count)
                
            # Zero the gradients before running the backward pass.
            model.zero_grad()

            # Backward pass: compute gradient of the loss with respect to all the learnable
            # parameters of the model. Internally, the parameters of each Module are stored
            # in Variables with requires_grad=True, so this call will compute gradients for
            # all learnable parameters in the model.
            loss.backward()

            # Update the weights using gradient descent. Each parameter is a Variable, so
            # we can access its data and gradients like we did before.
            for param in model.parameters():
                param.data -= learning_rate * param.grad.data
            #pdb.set_trace()
        print('epoch', epoch, 'loss', total_loss/13484000)

1911119.911328125
2012500.7702148438
1997849.4029947917
1961999.6116015625
2037776.5621171875
2058198.2413541668



KeyboardInterrupt: 

In [49]:
df_iter = pd.read_csv('data/yellow_tripdata_2009-01_cleaned.csv', chunksize=100000)
for df in tqdm(df_iter):
    x, y = df_to_torch(df)
    print(pd.Series((y - model(x).squeeze()).data.numpy()/60).describe())

count    100000.000000
mean         -1.218680
std          23.897057
min       -1432.373535
25%          -3.442153
50%          -0.994354
75%           1.088403
max        1430.219849
dtype: float64
count    100000.000000
mean         -1.089616
std          22.774660
min       -1443.842651
25%          -3.180760
50%          -0.903701
75%           1.134948
max        1431.973267
dtype: float64
count    100000.000000
mean         -1.097586
std          22.618757
min       -1439.093750
25%          -3.178469
50%          -0.913808
75%           1.155720
max        1432.233276
dtype: float64
count    100000.000000
mean         -1.014024
std          24.069305
min       -1439.317261
25%          -3.132402
50%          -0.884648
75%           1.154779
max        1430.691772
dtype: float64
count    100000.000000
mean         -1.044870
std          25.676497
min       -1439.896729
25%          -3.227267
50%          -0.904596
75%           1.140260
max        1430.439697
dtype: float64
count

KeyboardInterrupt: 

In [46]:
np.mean(stds)

20.490866306984547

In [37]:
pd.Series(((y - model(x).squeeze())/60).data.numpy()).describe()

count    10000.000000
mean         0.190011
std          7.169688
min        -25.187225
25%         -1.708399
50%         -0.226886
75%          1.536333
max        436.553894
dtype: float64

Variable containing:
-10200
[torch.FloatTensor of size 1]

In [31]:
pd.Series(((y - model(x).squeeze()) / 60).data.numpy()).describe()

count    10000.000000
mean        -0.138526
std         20.105570
min       -198.264816
25%         -2.602337
50%         -0.584000
75%          1.422255
max       1117.378784
dtype: float64

In [53]:
for param in model.parameters():
    print(param)

Parameter containing:
-8.7181e-02 -1.4039e-01  5.5304e-02  ...  -1.4271e-01 -7.6074e-02 -8.5804e-02
-2.1665e-02 -1.1186e-01  4.0417e-02  ...   1.5005e-01 -1.2282e-01 -5.4708e-02
 1.4068e-01 -5.6900e-02 -5.8072e-02  ...   7.2092e-02 -1.4730e-01  6.4929e-03
                ...                   ⋱                   ...                
 1.2554e-01 -8.7601e-02 -4.9987e-02  ...   1.3063e-01  1.2294e-01  7.8344e-02
 1.5451e-01 -3.6101e-03  1.5144e-01  ...  -8.1832e-02 -3.9803e-02 -1.3775e-01
 3.2955e-02 -1.1569e-01 -7.3116e-02  ...   8.0871e-02 -4.3422e-02 -1.1847e-02
[torch.FloatTensor of size 1000x36]

Parameter containing:
 2.5063e-02  2.7433e-02  9.6065e-03  ...  -6.6510e-03 -4.0478e-03  5.4308e-03
 2.2791e-02 -3.2669e-03  2.6060e-02  ...   3.1538e-02  2.0528e-02 -1.1208e-02
-1.8972e-02  3.1786e-02  1.7345e-02  ...   2.4548e-02 -2.0717e-02 -1.6142e-02
                ...                   ⋱                   ...                
 3.4726e-02 -2.2916e-03 -2.8864e-02  ...  -1.6556e-02 -2.5371